In [1]:
import pandas as pd
import numpy as np
# fancy graph
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
# i wanna see the diffrence
import seaborn as sns
# nlp
from sklearn.feature_extraction.text import TfidfVectorizer
from spacy.tokenizer import Tokenizer
import en_core_web_sm
# reproduce
np.random.seed(42)
SEED = 42


In [11]:
!ls

cleaned_data  graphs			project.ipynb  scraped_data
eda.ipynb     presentation_script.docx	README.md      testing.ipynb


<h2>Preapare Tokenizer and Corpus</h2>

In [2]:
import en_core_web_lg

In [3]:
nlp = en_core_web_lg.load()


In [4]:
tokenizer = Tokenizer(nlp.vocab)


In [5]:
tfidf = TfidfVectorizer(min_df=1,
                        max_df=0.95,
                        tokenizer=tokenizer)


## Loading The Data

In [22]:
df = pd.read_csv("../data/cleaned_data/chanellos.csv")
df.shape

(2088, 11)

In [23]:
df1 = pd.read_csv("../data/cleaned_data/calz.csv")
df1.shape

(1041, 13)

In [8]:
df.isnull().sum()

Unnamed: 0          0
time                0
customer_name       0
review_content    779
review_rating       0
store_name          0
store_rating        0
store_address       0
tot_num_rate        0
latitude            0
longitude           0
dtype: int64

In [13]:
df1.isnull().sum()

Unnamed: 0               0
time                     0
customer_name            0
review_content          84
review_rating            0
store_name               0
store_rating             0
store_address            0
num_to_rate              0
owner_response         937
owner_response_time    937
latitude                 0
longitude                0
dtype: int64

In [24]:
df = df.dropna()
df.shape

(1309, 11)

In [25]:
df1.drop("owner_response", axis=1, inplace=True)
df1.drop("owner_response_time", axis=1, inplace=True)
df1.dropna(inplace=True)
df1.shape

(957, 11)

In [27]:
df["label"] = [1] * df.shape[0]
df1["label"] = [0] * df1.shape[0]

In [28]:
df = pd.concat([df, df1])
del df1

In [33]:
df.head()

,Unnamed: 0,time,customer_name,review_content,review_rating,store_name,store_rating,store_address,tot_num_rate,latitude,longitude,label,num_to_rate
0,0,1.581293e+09,lauren sams,Great and fast service with excellent food.,5,Chanello's Pizza #33,3.1,1122 George Washington Hwy N,124.0,36.782,-76.338,1,NaN
2,2,1.550945e+09,Janel Brosseit,This rating goes more towards the food. Just r...,2,Chanello's Pizza #33,3.1,1122 George Washington Hwy N,124.0,36.782,-76.338,1,NaN
4,4,1.456272e+09,Alexandra Kehl,"Waited over an hour for our food, then the man...",1,Chanello's Pizza #33,3.1,1122 George Washington Hwy N,124.0,36.782,-76.338,1,NaN
5,5,1.550945e+09,Big A,"No thanks owner, u only get so many chances, d...",1,Chanello's Pizza #33,3.1,1122 George Washington Hwy N,124.0,36.782,-76.338,1,NaN
6,6,1.582243e+09,Brandon Davis,"Friendly staff, always willing to help a custo...",5,Chanello's Pizza #33,3.1,1122 George Washington Hwy N,124.0,36.782,-76.338,1,NaN


## Split training and testing data

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(df)
X_train.shape, X_test.shape

((1699, 13), (567, 13))

In [36]:
# looking for what makes a good review
df['target'] = df['review_rating'] >= 3

# looking at the oposite thus the anti
df['anti_target'] = df['review_rating'] <= 3

# changing bools to ints
df['target'] = df['target'].map({True: 1, False: 0})


In [37]:
# baseline off the shelf model
tfidf.fit(X_train.review_content)


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.95, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<spacy.tokenizer.Tokenizer object at 0x7f730dd47bd0>,
                use_idf=True, vocabulary=None)

In [38]:
# make vector encodings of the data from the chanellos dataset
X_train_vect = tfidf.transform(X_train)
X_test_vect = tfidf.transform(X_test)


In [39]:
# make sparse matrixs from the vectorizations
dtm_train = X_train_vect.todense()
dtm_test = X_test_vect.todense()


In [40]:
from sklearn.neighbors import NearestNeighbors


In [41]:
nn = NearestNeighbors(n_jobs=-1)
nn.fit(dtm_train)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=-1, n_neighbors=5, p=2, radius=1.0)

In [44]:
nn.kneighbors(tfidf.transform(["pretty good would recommend"]).todense())[1]

array([[1, 4, 3, 0, 2]])

In [50]:
df.label.value_counts(normalize=True)

1    0.57767
0    0.42233
Name: label, dtype: float64